In [1]:
# Author: Arthur Prigent
# Email: aprigent@ictp.it

#### DATA ####
# https://downloads.psl.noaa.gov/Datasets/
from functions import *
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image

2024-04-01 17:14:42.361533


In [4]:
path_ersstv5 = 'https://psl.noaa.gov/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc'
(ssta_atl3_ersst,ssta_aba_ersst,
 ssta_nino34_ersst,ssta_dni_ersst,
 ssta_cni_ersst,ssta_nni_ersst,iod_ersst) = read_data_compute_anomalies_ersstv5_for_mail(path_ersstv5)

In [8]:
ssta_atl3_norm_ersst = ssta_atl3_ersst/ssta_atl3_ersst.std(dim='time')
ssta_aba_norm_ersst = ssta_aba_ersst/ssta_aba_ersst.std(dim='time')
ssta_nino34_norm_ersst = ssta_nino34_ersst/ssta_nino34_ersst.std(dim='time')
ssta_dni_norm_ersst = ssta_dni_ersst/ssta_dni_ersst.std(dim='time')



In [9]:
last_timesteps = [str(ssta_atl3_ersst.time[-3].values)[:7],
str(ssta_atl3_ersst.time[-2].values)[:7],
str(ssta_atl3_ersst.time[-1].values)[:7]]



In [10]:
import os
mail_sender = os.environ.get("USERNAME")
password = os.environ.get("PASSWORD")



In [7]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Email configuration
sender_email = mail_sender
sender_password = password
smtp_server = "smtp.gmail.com"
smtp_port = 587

# List of recipient email addresses
recipient_emails = ["aprigent@ictp.it"] #"ahans@geomar.de"

# Create the email message
subject = "Tropical Atlantic Variability update"
body = """
Hello!

Please find at the following link a little update on the current state of the tropics for the last three timesteps: {last_timesteps}

Last three standardized anomalies from ERSSTv5:
ATL3 ({std_atl3} degC): {ssta_atl3_norm_ersst} degC
ABA ({std_aba} degC): {ssta_aba_norm_ersst} degC
NINO3.4 ({std_nino34} degC): {ssta_nino34_norm_ersst} degC
DNI ({std_dni} degC): {ssta_dni_norm_ersst} degC
IOD : {iod_ersst} degC

Best regards,
Arthur Prigent

https://aprig.github.io/ATL3_ABA_SSTa/automatic_time_series_SST.html
"""


# Connect to the SMTP server
with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()
    # Login to your Gmail account
    server.login(sender_email, sender_password)
    
    for recipient_email in recipient_emails:
        # Create the email message
        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = recipient_email
        message['Subject'] = subject

        formatted_body = body.format(
            last_timesteps=last_timesteps,
            std_atl3 = np.round(ssta_atl3_ersst.std(dim='time'),2).values,
            std_aba = np.round(ssta_aba_ersst.std(dim='time'),2).values,
            std_nino34 = np.round(ssta_nino34_ersst.std(dim='time'),2).values,
            std_dni = np.round(ssta_dni_ersst.std(dim='time'),2).values,
            ssta_atl3_norm_ersst=np.round(ssta_atl3_ersst[-3:],2).values,
            ssta_aba_norm_ersst=np.round(ssta_aba_ersst[-3:],2).values,
            ssta_nino34_norm_ersst=np.round(ssta_nino34_ersst[-3:],2).values,
            ssta_dni_norm_ersst=np.round(ssta_dni_ersst[-3:],2).values,
            iod_ersst=np.round(iod_ersst[-3:],2).values
            
        )
        message.attach(MIMEText(formatted_body, 'plain'))

        # Send the email
        server.sendmail(sender_email, recipient_email, message.as_string())

# Quit the server
print("Emails sent successfully!")


UnicodeEncodeError: 'ascii' codec can't encode character '\xa0' in position 32: ordinal not in range(128)